### Actual calculator

In [ ]:
import sys
from pathlib import Path

root = Path().resolve().parent  # go up one level from notebook’s folder
sys.path.append(str(root))

import calculators.ToyModels as toy_models
from calculators.Training import Training
import pandas as pd
import torch

# Set device for PyTorch
device = torch.device("mps")
print(f"Using device: {device}")

######################################## Data Preprocessing ########################################
PREPROCESSING_TYPE = "normalization" # set the preprocessing to apply to the data options: normalization, min_max_scaling, GADF

REAL_INPUT_SIZE = 10 ## CHANGE SIZE HERE

######################################## Training ########################################
MODEL_NAME ="SimpleCNN" # set which model to use examples: KAN, resnet18, baichuan-inc/Baichuan-13B-Chat
NUM_EPOCHS = 50
BATCH_SIZE = 32
# INPUT_SIZE = (1, 3, 224, 224) # 4d input for resnet # (1,128) # batch, max_seq_length for llm
if MODEL_NAME == "SimpleCNN":
    INPUT_SIZE = (1, 1, REAL_INPUT_SIZE) # 4d input for resnet # (1,128) # batch, max_seq_length for llm
else:
    INPUT_SIZE = (1, REAL_INPUT_SIZE) # 

EVALUATION_STRATEGY = "cross_validation" # set the evaluation strategy options: cross_validation, train_test_split
K_FOLDS = 5 # Only used if EVALUATION_STRATEGY is cross_validation
SPLIT_RATIO = 0.8 # Only used if EVALUATION_STRATEGY is train_test_split

NUM_LAYERS = 2
GRID_SIZE = 5
NUM_CLASSES = 2
DIN = REAL_INPUT_SIZE
DOUT = 2

# Transformer specific parameters
CONTEXT_LENGTH = REAL_INPUT_SIZE  
EMBEDDING_SIZE = 16  
NUM_HEADS = 2       
NUM_DECODER_BLOCKS = 3  
FEED_FORWARD_SIZE = 64  

######################################## Inference ########################################
NUM_INFERENCES = 10000 # number of inferences to run


# GENERAL CONFIG
NUM_SAMPLES = 256 # number of samples that are used to calculate the energy consumption
FLOAT_PRECISION = 32 # number of bits used to represent a floating point number
PROCESSOR_FLOPS_PER_SECOND = 3e12 # theoretical maximum number of floating point operations per second for the processor
PROCESSOR_MAX_POWER = 10 # maximum power consumption of the processor in Watts

# Timeseries specific
SAMPLE_SIZE = REAL_INPUT_SIZE # size of a single sample e.g. number of timesteps in a timeseries or number of pixels in an image


NUM_LAYERS = [3,4,5,6,7] # K
HIDDEN_SIZE = [10] # H
energy_consumption  ={}
model_names = ['MLP_practical', 'CNN_practical' ,'KAN', 'Transformer']
calculator = None
for model_name in model_names:
    curr_model = {}
    for L in NUM_LAYERS:
        for hs in HIDDEN_SIZE:
            #INPUT_SIZE = (1, 1, 10) #is this ok for all models?
            print(f"Calculating energy consumption for {model_name} with {L} layers and hidden size {hs}")
            calculator = None
            if model_name == "MLP_practical":
                model = toy_models.SimpleMLP_practical(input_size=REAL_INPUT_SIZE, hidden_size=hs, 
                                                       output_size=DOUT, num_layers=L).to(device)
            elif model_name == "CNN_practical":
                model = toy_models.SimpleCNN_practical(output_size=DOUT, hidden_channels=1, 
                                                       num_layers=L).to(device)
            elif model_name == "KAN":
                model = toy_models.KANLikeRegressor(
                    input_dim=REAL_INPUT_SIZE,
                    num_hidden_layers_per_feature_fn=L,  # Varying sub-layers
                    nodes_per_layer_in_sub_fn=hs
                ).to(device)
                INPUT_SIZE = (1, REAL_INPUT_SIZE)
            elif model_name == "Transformer":  # Changed from "SimpleTransformer" to "Transformer"
                model = toy_models.Transformer(
                    num_tokens=CONTEXT_LENGTH,
                    num_token_vals=20, #nodes per layer
                    num_emb=hs, #* 2,
                    num_neurons=hs,
                    num_heads=NUM_HEADS,
                    num_blocks=L,
                    device=device
            ).to(device)
            training = Training(
                model_name=model,
                num_epochs=NUM_EPOCHS,
                batch_size=BATCH_SIZE,
                processor_flops_per_second=PROCESSOR_FLOPS_PER_SECOND,
                processor_max_power=PROCESSOR_MAX_POWER,
                num_samples=NUM_SAMPLES,
                input_size=INPUT_SIZE,
                evaluation_strategy=EVALUATION_STRATEGY,
                k_folds=K_FOLDS,
                split_ratio=SPLIT_RATIO,
                calculator=calculator
            )
            if model_name == "MLP":
                model_str = "MLP"
            elif model_name == "CNN":
                model_str = "CNN"
            elif model_name == "Transformer":
                model_str = "Transformer"
            else:
                model_str = model_name

            curr_model[(L,hs)] = training.calculate_energy()['training_energy']
    energy_consumption[model_str] = curr_model


import pandas as pd
df = pd.DataFrame(energy_consumption)
df.to_csv('energy_consumption_models_small_calflops.csv')
df

Using device: mps
Calculating energy consumption for MLP_practical with 3 layers and hidden size 10
Calculating energy consumption for MLP_practical with 4 layers and hidden size 10
Calculating energy consumption for MLP_practical with 5 layers and hidden size 10
Calculating energy consumption for MLP_practical with 6 layers and hidden size 10
Calculating energy consumption for MLP_practical with 7 layers and hidden size 10
Calculating energy consumption for CNN_practical with 3 layers and hidden size 10
Calculating energy consumption for CNN_practical with 4 layers and hidden size 10
Calculating energy consumption for CNN_practical with 5 layers and hidden size 10
Calculating energy consumption for CNN_practical with 6 layers and hidden size 10
Calculating energy consumption for CNN_practical with 7 layers and hidden size 10
Calculating energy consumption for KAN with 3 layers and hidden size 10
Calculating energy consumption for KAN with 4 layers and hidden size 10
Calculating energy

,,MLP_practical,CNN_practical,KAN,Transformer
3,10,0.001178,0.002550,0.012032,0.448000
4,10,0.001715,0.008428,0.017408,0.596992
5,10,0.002253,0.030013,0.022784,0.745984
6,10,0.002790,0.112507,0.028160,0.894976
7,10,0.003328,0.434780,0.033536,1.043968
